# Train a Tokenizer

### Import data

I'm using **Farsi** and **English** corpus datasets to have a tokenizer that could handle both `en` and `fa` words in a texts. I choose this methode so if you want to train the relevant model with your data, you can have both `fa` and `en` data in your dataset.

In [1]:
from datasets import load_dataset, concatenate_datasets
import pandas as pd

en_dataset = load_dataset("tiennv/english-wiki-corpus")
fa_dataset = load_dataset("RohanAiLab/persian_blog")

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [2]:
# Merge two datasets (Farsi and English)
full_dataset = concatenate_datasets([en_dataset["train"], fa_dataset["train"]])
full_dataset

Dataset({
    features: ['text'],
    num_rows: 11072070
})

### Convert `datasets` obj to `pandas` obj to do preprocessing in batches and then save each batch as a CSV file

In [3]:
import re
pattern = r'[^A-Za-zآ-ی0-9]+'
pattern = re.compile(pattern)

def normalize_text(text, pattern=pattern):
    text = str(text).lower()
    text = re.sub(pattern, " ", text)
    return text


In [4]:
import tqdm
BATCH_SIZE = 100000  # Adjust this to your system's memory constraints

for i, df in tqdm.tqdm(enumerate(full_dataset.to_pandas(batched=True, batch_size=BATCH_SIZE))):
    df = df.dropna().drop_duplicates()
    df["text"] = df["text"].apply(normalize_text)
    df.to_csv(f"data/data_{i+1}.csv", index=False)
    del df

111it [02:58,  1.61s/it]


### Loading all CSV files as a `datasets` obj to use them for training tokenizer

In [1]:
from datasets import load_dataset
from glob import glob

# gather path of all CSV files
csv_path = glob("data/*.csv")

raw_dataset = load_dataset("csv", data_files=csv_path, keep_in_memory=False) # set `keep_in_memory = True` to increase training speed if you have enough RAM capacity
raw_dataset

Resolving data files:   0%|          | 0/111 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3246583
    })
})

In [2]:
raw_dataset.column_names

{'train': ['text']}

In [10]:
print(raw_dataset["train"][20]["text"])
print(raw_dataset["train"][300100]["text"])

my basic philosophy is that every case is an impact case that every case in some incremental fashion every success every life saved every person got out of immediate danger every person who s freed from the threat of jail advances that larger process of change in some fashion you are going to be dealing with the one person whose story opens up the stories of 10 20 100 1 000 others my feeling is always that by getting that individual s story out you can illuminate the lives of others and help them but if you can t do something for that one person then you re not ultimately going to be able to do much effectively for the 1 000 the 10 000 the million 
موفقیت وابسته به شانس است یا کار سخت از بچگی و توی خیلی از فیلم سریال کتاب و صحبت افراد موفق دیدیم که دلیل موفقیت خودشون رو کار سخت و پشتکار معرفی کردند اما آیا واقعا این حرف ها واقعیت داره توی این مقاله قراره دقیق تر و علمی تر به این کلیشه مهم بپردازیم پس حتما تا جایی که حوصلتون میکشه با من همراه باشید خب اول بهتره از این دو رقیب دیرینه تعر

Create a generator object to batch data and use them for tokenizer training

### Importing the tokenizr

In [11]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [25]:
# Check the tokenizer
example = raw_dataset["train"][300100]["text"]
print(example)

tokens = old_tokenizer.tokenize(example)
tokens

['Ùħ',
 'ÙĪÙģÙĤ',
 'ÛĮØª',
 'ĠÙĪØ§Ø¨',
 'Ø³ØªÙĩ',
 'ĠØ¨Ùĩ',
 'ĠØ´',
 'Ø§ÙĨØ³',
 'ĠØ§Ø³Øª',
 'ĠÛĮØ§',
 'ĠÚ©Ø§Ø±',
 'ĠØ³Ø®Øª',
 'ĠØ§Ø²',
 'ĠØ¨ÚĨ',
 'Ú¯ÛĮ',
 'ĠÙĪ',
 'ĠØªÙĪ',
 'ÛĮ',
 'ĠØ®ÛĮ',
 'ÙĦÛĮ',
 'ĠØ§Ø²',
 'ĠÙģÛĮ',
 'ÙĦÙħ',
 'ĠØ³Ø±ÛĮ',
 'Ø§ÙĦ',
 'ĠÚ©ØªØ§Ø¨',
 'ĠÙĪ',
 'ĠØµØŃ',
 'Ø¨Øª',
 'ĠØ§ÙģØ±Ø§Ø¯',
 'ĠÙħ',
 'ÙĪÙģÙĤ',
 'ĠØ¯ÛĮØ¯',
 'ÛĮÙħ',
 'ĠÚ©',
 'Ùĩ',
 'ĠØ¯ÙĦÛĮÙĦ',
 'ĠÙħ',
 'ÙĪÙģÙĤ',
 'ÛĮØª',
 'ĠØ®ÙĪØ¯',
 'Ø´ÙĪÙĨ',
 'ĠØ±ÙĪ',
 'ĠÚ©Ø§Ø±',
 'ĠØ³Ø®Øª',
 'ĠÙĪ',
 'ĠÙ¾Ø´Øª',
 'Ú©Ø§Ø±',
 'ĠÙħØ¹Ø±Ùģ',
 'ÛĮ',
 'ĠÚ©Ø±Ø¯',
 'ÙĨØ¯',
 'ĠØ§ÙħØ§',
 'ĠØ¢ÛĮØ§',
 'ĠÙĪØ§ÙĤ',
 'Ø¹Ø§',
 'ĠØ§ÛĮÙĨ',
 'ĠØŃØ±Ùģ',
 'ĠÙĩØ§',
 'ĠÙĪØ§ÙĤØ¹',
 'ÛĮØª',
 'ĠØ¯Ø§Ø±',
 'Ùĩ',
 'ĠØªÙĪ',
 'ÛĮ',
 'ĠØ§ÛĮÙĨ',
 'ĠÙħÙĤØ§ÙĦ',
 'Ùĩ',
 'ĠÙĤØ±Ø§Ø±',
 'Ùĩ',
 'ĠØ¯',
 'ÙĤÛĮ',
 'ÙĤ',
 'ĠØªØ±',
 'ĠÙĪ',
 'ĠØ¹ÙĦÙħÛĮ',
 'ĠØªØ±',
 'ĠØ¨Ùĩ',
 'ĠØ§ÛĮÙĨ',
 'ĠÚ©ÙĦÛĮ',
 'Ø´Ùĩ',
 'ĠÙħÙĩÙħ',
 'ĠØ¨',
 'Ù¾Ø±Ø¯',
 'Ø§Ø²',
 'ÛĮÙħ',
 'ĠÙ¾Ø³',
 'ĠØŃØª',
 'ÙħØ§',
 'ĠØªØ§',
 'ĠØ¬Ø§',
 'ÛĮ',
 'ÛĮ',
 'ĠÚ©',
 'Ùĩ',
 'ĠØŃÙĪ',
 'ØµÙĦØª',
 'ÙĪÙĨ',
 'ĠÙħÛĮÚ©'

### Start training and save new tokenizer

Create a generative object to load data samples in bathces while training tokenizer

In [13]:
NUM_CHUNK = 1000
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), NUM_CHUNK):
        samples = dataset[start_idx : start_idx + NUM_CHUNK]
        yield samples["text"]
training_corpus = get_training_corpus()

In [14]:
# Start training - taking a while (depending on your CPU)
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 200000)  # `200,000` is the vocab_size for new tokenizer
tokenizer.save_pretrained("./tokenizer")

('./final_output\\tokenizer_config.json',
 './final_output\\special_tokens_map.json',
 './final_output\\tokenizer.json')

### Test new tokenizer

In [15]:
print(example)
new_tokens = tokenizer.tokenize(example)
new_tokens

موفقیت وابسته به شانس است یا کار سخت از بچگی و توی خیلی از فیلم سریال کتاب و صحبت افراد موفق دیدیم که دلیل موفقیت خودشون رو کار سخت و پشتکار معرفی کردند اما آیا واقعا این حرف ها واقعیت داره توی این مقاله قراره دقیق تر و علمی تر به این کلیشه مهم بپردازیم پس حتما تا جایی که حوصلتون میکشه با من همراه باشید خب اول بهتره از این دو رقیب دیرینه تعریفی داشته باشیم چیکار کنیم شانس چیست شانس بین آدم ها تعاریف مختلفی داره ولی بطور کلی میشه شانس رو اتفاق یا پیشامدی بیان کرد که بدون دلیل اتفاق می افته و در زندگی یک موجودی تاثیر مثبت یا منفی میذاره از این رو خوش شانسی و بد شانسی رو میشه بر اساس تاثیر این اتفاق ها بیان کرد لوک خوش شانسکار سخت چیست واقعا لازمه این رو هم توضیح بدم فک کنم دقیقا 100 درصدتون که در حال خوندن هستین با معنی کار سخت آشنایید تاثیر انکار ناپذیر شانسبا تعریف چند داستان و اتفاق جالب میخوام نظر افرادی رو تغییر بدم که موفقیت رو فقط حاصل تلاش و کار سخت میدونن مالاریا و جنگدر 14 امین سال جنگ ویتنام و آمریکا که چین حامی ویتنام بود زنی به نام یویو تو سرپرست گروهی از محققین شده بود که م

['ÙħÙĪÙģÙĤÛĮØª',
 'ĠÙĪØ§Ø¨Ø³ØªÙĩ',
 'ĠØ¨Ùĩ',
 'ĠØ´Ø§ÙĨØ³',
 'ĠØ§Ø³Øª',
 'ĠÛĮØ§',
 'ĠÚ©Ø§Ø±',
 'ĠØ³Ø®Øª',
 'ĠØ§Ø²',
 'ĠØ¨ÚĨÚ¯ÛĮ',
 'ĠÙĪ',
 'ĠØªÙĪÛĮ',
 'ĠØ®ÛĮÙĦÛĮ',
 'ĠØ§Ø²',
 'ĠÙģÛĮÙĦÙħ',
 'ĠØ³Ø±ÛĮØ§ÙĦ',
 'ĠÚ©ØªØ§Ø¨',
 'ĠÙĪ',
 'ĠØµØŃØ¨Øª',
 'ĠØ§ÙģØ±Ø§Ø¯',
 'ĠÙħÙĪÙģÙĤ',
 'ĠØ¯ÛĮØ¯ÛĮÙħ',
 'ĠÚ©Ùĩ',
 'ĠØ¯ÙĦÛĮÙĦ',
 'ĠÙħÙĪÙģÙĤÛĮØª',
 'ĠØ®ÙĪØ¯Ø´ÙĪÙĨ',
 'ĠØ±ÙĪ',
 'ĠÚ©Ø§Ø±',
 'ĠØ³Ø®Øª',
 'ĠÙĪ',
 'ĠÙ¾Ø´ØªÚ©Ø§Ø±',
 'ĠÙħØ¹Ø±ÙģÛĮ',
 'ĠÚ©Ø±Ø¯ÙĨØ¯',
 'ĠØ§ÙħØ§',
 'ĠØ¢ÛĮØ§',
 'ĠÙĪØ§ÙĤØ¹Ø§',
 'ĠØ§ÛĮÙĨ',
 'ĠØŃØ±Ùģ',
 'ĠÙĩØ§',
 'ĠÙĪØ§ÙĤØ¹ÛĮØª',
 'ĠØ¯Ø§Ø±Ùĩ',
 'ĠØªÙĪÛĮ',
 'ĠØ§ÛĮÙĨ',
 'ĠÙħÙĤØ§ÙĦÙĩ',
 'ĠÙĤØ±Ø§Ø±Ùĩ',
 'ĠØ¯ÙĤÛĮÙĤ',
 'ĠØªØ±',
 'ĠÙĪ',
 'ĠØ¹ÙĦÙħÛĮ',
 'ĠØªØ±',
 'ĠØ¨Ùĩ',
 'ĠØ§ÛĮÙĨ',
 'ĠÚ©ÙĦÛĮØ´Ùĩ',
 'ĠÙħÙĩÙħ',
 'ĠØ¨Ù¾Ø±Ø¯Ø§Ø²ÛĮÙħ',
 'ĠÙ¾Ø³',
 'ĠØŃØªÙħØ§',
 'ĠØªØ§',
 'ĠØ¬Ø§ÛĮÛĮ',
 'ĠÚ©Ùĩ',
 'ĠØŃÙĪØµ',
 'ÙĦØªÙĪÙĨ',
 'ĠÙħÛĮÚ©Ø´Ùĩ',
 'ĠØ¨Ø§',
 'ĠÙħÙĨ',
 'ĠÙĩÙħØ±Ø§Ùĩ',
 'ĠØ¨Ø§Ø´ÛĮØ¯',
 'ĠØ®Ø¨',
 'ĠØ§ÙĪÙĦ',
 'ĠØ¨ÙĩØªØ±Ùĩ',
 'ĠØ§Ø²',
 'ĠØ§ÛĮÙĨ',
 'ĠØ¯ÙĪ',
 'ĠØ±ÙĤÛĮØ¨',
 'ĠØ¯ÛĮØ±ÛĮÙĨÙĩ',
 'ĠØªØ¹Ø±ÛĮÙģÛĮ',
 '

In [16]:
# Convert tokens to string
tokenizer.convert_tokens_to_string(new_tokens)

'موفقیت وابسته به شانس است یا کار سخت از بچگی و توی خیلی از فیلم سریال کتاب و صحبت افراد موفق دیدیم که دلیل موفقیت خودشون رو کار سخت و پشتکار معرفی کردند اما آیا واقعا این حرف ها واقعیت داره توی این مقاله قراره دقیق تر و علمی تر به این کلیشه مهم بپردازیم پس حتما تا جایی که حوصلتون میکشه با من همراه باشید خب اول بهتره از این دو رقیب دیرینه تعریفی داشته باشیم چیکار کنیم شانس چیست شانس بین آدم ها تعاریف مختلفی داره ولی بطور کلی میشه شانس رو اتفاق یا پیشامدی بیان کرد که بدون دلیل اتفاق می افته و در زندگی یک موجودی تاثیر مثبت یا منفی میذاره از این رو خوش شانسی و بد شانسی رو میشه بر اساس تاثیر این اتفاق ها بیان کرد لوک خوش شانسکار سخت چیست واقعا لازمه این رو هم توضیح بدم فک کنم دقیقا 100 درصدتون که در حال خوندن هستین با معنی کار سخت آشنایید تاثیر انکار ناپذیر شانسبا تعریف چند داستان و اتفاق جالب میخوام نظر افرادی رو تغییر بدم که موفقیت رو فقط حاصل تلاش و کار سخت میدونن مالاریا و جنگدر 14 امین سال جنگ ویتنام و آمریکا که چین حامی ویتنام بود زنی به نام یویو تو سرپرست گروهی از محققین شده بود که 

In [17]:
example2 = raw_dataset["train"][20]["text"]

old_tokens = old_tokenizer.tokenize(example2)
old_tokens

['my',
 'Ġbasic',
 'Ġphilosophy',
 'Ġis',
 'Ġthat',
 'Ġevery',
 'Ġcase',
 'Ġis',
 'Ġan',
 'Ġimpact',
 'Ġcase',
 'Ġthat',
 'Ġevery',
 'Ġcase',
 'Ġin',
 'Ġsome',
 'Ġincremental',
 'Ġfashion',
 'Ġevery',
 'Ġsuccess',
 'Ġevery',
 'Ġlife',
 'Ġsaved',
 'Ġevery',
 'Ġperson',
 'Ġgot',
 'Ġout',
 'Ġof',
 'Ġimmediate',
 'Ġdanger',
 'Ġevery',
 'Ġperson',
 'Ġwho',
 'Ġs',
 'Ġfreed',
 'Ġfrom',
 'Ġthe',
 'Ġthreat',
 'Ġof',
 'Ġjail',
 'Ġadvances',
 'Ġthat',
 'Ġlarger',
 'Ġprocess',
 'Ġof',
 'Ġchange',
 'Ġin',
 'Ġsome',
 'Ġfashion',
 'Ġyou',
 'Ġare',
 'Ġgoing',
 'Ġto',
 'Ġbe',
 'Ġdealing',
 'Ġwith',
 'Ġthe',
 'Ġone',
 'Ġperson',
 'Ġwhose',
 'Ġstory',
 'Ġopens',
 'Ġup',
 'Ġthe',
 'Ġstories',
 'Ġof',
 'Ġ10',
 'Ġ20',
 'Ġ100',
 'Ġ1',
 'Ġ000',
 'Ġothers',
 'Ġmy',
 'Ġfeeling',
 'Ġis',
 'Ġalways',
 'Ġthat',
 'Ġby',
 'Ġgetting',
 'Ġthat',
 'Ġindividual',
 'Ġs',
 'Ġstory',
 'Ġout',
 'Ġyou',
 'Ġcan',
 'Ġillumin',
 'ate',
 'Ġthe',
 'Ġlives',
 'Ġof',
 'Ġothers',
 'Ġand',
 'Ġhelp',
 'Ġthem',
 'Ġbut',
 'Ġif',
 'Ġyou'

In [18]:
new_tokens = tokenizer.tokenize(example2)
new_tokens

['my',
 'Ġbasic',
 'Ġphilosophy',
 'Ġis',
 'Ġthat',
 'Ġevery',
 'Ġcase',
 'Ġis',
 'Ġan',
 'Ġimpact',
 'Ġcase',
 'Ġthat',
 'Ġevery',
 'Ġcase',
 'Ġin',
 'Ġsome',
 'Ġincremental',
 'Ġfashion',
 'Ġevery',
 'Ġsuccess',
 'Ġevery',
 'Ġlife',
 'Ġsaved',
 'Ġevery',
 'Ġperson',
 'Ġgot',
 'Ġout',
 'Ġof',
 'Ġimmediate',
 'Ġdanger',
 'Ġevery',
 'Ġperson',
 'Ġwho',
 'Ġs',
 'Ġfreed',
 'Ġfrom',
 'Ġthe',
 'Ġthreat',
 'Ġof',
 'Ġjail',
 'Ġadvances',
 'Ġthat',
 'Ġlarger',
 'Ġprocess',
 'Ġof',
 'Ġchange',
 'Ġin',
 'Ġsome',
 'Ġfashion',
 'Ġyou',
 'Ġare',
 'Ġgoing',
 'Ġto',
 'Ġbe',
 'Ġdealing',
 'Ġwith',
 'Ġthe',
 'Ġone',
 'Ġperson',
 'Ġwhose',
 'Ġstory',
 'Ġopens',
 'Ġup',
 'Ġthe',
 'Ġstories',
 'Ġof',
 'Ġ10',
 'Ġ20',
 'Ġ100',
 'Ġ1',
 'Ġ000',
 'Ġothers',
 'Ġmy',
 'Ġfeeling',
 'Ġis',
 'Ġalways',
 'Ġthat',
 'Ġby',
 'Ġgetting',
 'Ġthat',
 'Ġindividual',
 'Ġs',
 'Ġstory',
 'Ġout',
 'Ġyou',
 'Ġcan',
 'Ġilluminate',
 'Ġthe',
 'Ġlives',
 'Ġof',
 'Ġothers',
 'Ġand',
 'Ġhelp',
 'Ġthem',
 'Ġbut',
 'Ġif',
 'Ġyou',
 'Ġ

In [19]:
tokenizer.convert_tokens_to_string(new_tokens)

'my basic philosophy is that every case is an impact case that every case in some incremental fashion every success every life saved every person got out of immediate danger every person who s freed from the threat of jail advances that larger process of change in some fashion you are going to be dealing with the one person whose story opens up the stories of 10 20 100 1 000 others my feeling is always that by getting that individual s story out you can illuminate the lives of others and help them but if you can t do something for that one person then you re not ultimately going to be able to do much effectively for the 1 000 the 10 000 the million '

### Load the new tokenizer and upload it to 🤗Hub

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
tokenizer.push_to_hub(repo_id="ali619/bigscience-bloom-1b7-finetune-Summarization-Persian-News", commit_message="Update tokenizer with fa-en corpus dataset")

README.md:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ali619/bigscience-bloom-1b7-finetune-Summarization-Persian-News/commit/42655c6d75826ac9dfc6b872c504dad9c7b59700', commit_message='Update tokenizer with fa-en corpus dataset', commit_description='', oid='42655c6d75826ac9dfc6b872c504dad9c7b59700', pr_url=None, pr_revision=None, pr_num=None)